In [1]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
from geopy.distance import geodesic
import random
import datetime
# from datetime import datetime, timedelta, strftime  # Импортируйте strftime
import warnings
warnings.filterwarnings('ignore')

In [2]:
# adresses = pd.read_csv('london_postcodes-ons-postcodes-directory-feb22.csv', delimiter=',')
adresses = pd.read_csv('London postcodes.csv', delimiter=',')

adresses.head()

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,GridRef,County,District,Ward,...,Population,Households,Built up area,Built up sub-division,Lower layer super output area,Rural/urban,Region,Altitude,London zone,LSOA Code
0,BR1 1AA,Yes,51.401546,0.015415,540291,168873,TQ402688,Greater London,Bromley,Bromley Town,...,NaN,NaN,Greater London,Bromley,Bromley 018B,Urban major conurbation,London,71,5.0,E01000675
1,BR1 1AB,Yes,51.406333,0.015208,540262,169405,TQ402694,Greater London,Bromley,Bromley Town,...,NaN,NaN,Greater London,Bromley,Bromley 008B,Urban major conurbation,London,71,4.0,E01000676
2,BR1 1AD,Yes,51.400057,0.016715,540386,168710,TQ403687,Greater London,Bromley,Bromley Town,...,NaN,NaN,Greater London,Bromley,Bromley 018B,Urban major conurbation,London,53,5.0,E01000675
3,BR1 1AE,Yes,51.404543,0.014195,540197,169204,TQ401692,Greater London,Bromley,Bromley Town,...,34.0,21.0,Greater London,Bromley,Bromley 018C,Urban major conurbation,London,71,4.0,E01000677
4,BR1 1AF,Yes,51.401392,0.014948,540259,168855,TQ402688,Greater London,Bromley,Bromley Town,...,NaN,NaN,Greater London,Bromley,Bromley 018B,Urban major conurbation,London,58,5.0,E01000675


In [3]:
taxi_rides = pd.DataFrame(
    columns=['driver_id', 'client_id', 'start_point', 'end_point', 'distance', 'start_time',
             'end_time', 'road_time', 'cost', 'driver_rate', 'client_rate', 'category_driver_feedback', 
             'text_driver_feedback', 'category_client_feedback', 'text_client_feedback'
             ]
    )

In [4]:
num_rows = 10_000_000
num_drivers = 5000
num_clients = 2000

start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2024, 1, 1)


driver_id_range = range(1, num_drivers + 1)
client_id_range = range(1, num_clients + 1)


random_addresses = adresses.sample(n=num_rows, random_state=42, replace=True)

# Заполняем столбцы start_point и end_point случайными парами широты и долготы
taxi_rides['start_point'] = list(zip(random_addresses['Latitude'], random_addresses['Longitude']))
taxi_rides['end_point'] = list(zip(random_addresses['Latitude'].sample(frac=1, random_state=42), random_addresses['Longitude'].sample(frac=1, random_state=42)))

taxi_rides['driver_id'] = [random.choice(driver_id_range) for i in range(taxi_rides.shape[0])]
taxi_rides['client_id'] = [random.choice(client_id_range) for i in range(taxi_rides.shape[0])]

# Рассчитываем расстояние
taxi_rides['distance'] = [geodesic(start, end).km for start, end in taxi_rides[['start_point', 'end_point']].values]
taxi_rides['distance'] = taxi_rides['distance'].round(2)

taxi_rides['start_time'] = [datetime.datetime.strftime(start_date + datetime.timedelta(seconds=random.randint(0, 31536000)), '%Y-%m-%d %H:%M:%S') for i in range(num_rows)]
taxi_rides['road_time'] = [datetime.timedelta(seconds=distance / 50 * 3600) for distance in taxi_rides['distance']]
taxi_rides['end_time'] = [datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + road_time for start_time, road_time in zip(taxi_rides['start_time'], taxi_rides['road_time'])]

taxi_rides['start_time'] = pd.to_datetime(taxi_rides['start_time'], format='%Y-%m-%d %H:%M:%S')
taxi_rides['end_time'] = pd.to_datetime(taxi_rides['end_time'], format='%Y-%m-%d %H:%M:%S')


In [5]:
def count_cost(start_time, distance):
    cost = 5 + .5 * distance
    if (start_time.hour >= 8 and start_time.hour <= 11) or (start_time.hour >= 17 and start_time.hour <= 19):
        cost *= 1.5

    if (start_time.hour >= 23 and start_time.hour <= 6):
        cost *= 1.75
    return cost


taxi_rides['cost'] = [count_cost(time, dist) for time, dist in zip(taxi_rides['start_time'], taxi_rides['distance'])]
taxi_rides['cost'] = taxi_rides['cost'].round(2)

In [6]:
driver_rate_idx = np.random.randint(low=0, high=num_drivers, size=int(num_rows*0.45))
driver_rate_distr_arr = np.random.multinomial(1, [.2, .05, .1, .25, .4], size=int(num_rows*.45))
taxi_rides['driver_rate'][driver_rate_idx] = np.where(driver_rate_distr_arr == 1)[1] + 1

client_rate_idx = np.random.randint(low=0, high=num_drivers, size=int(num_rows*0.6))
client_rate_distr_arr = np.random.multinomial(1, [.2, .05, .1, .25, .4], size=int(num_rows*0.6))
taxi_rides['client_rate'][driver_rate_idx] = np.where(driver_rate_distr_arr == 1)[1] + 1

In [7]:
driver_fb_cat_good = ['good service', 'nice car', 'expert navigation']
driver_fb_cat_bad = ['awful service', 'bad car', 'dirty', 'rude']

client_fb_cat_good = ['polite', 'pleasant', 'quiet']
client_fb_cat_bad = ['unpolite', 'unpleasant', 'loud', 'dirty']

category_driver_good_feedback_idx = np.random.choice(taxi_rides[taxi_rides.driver_rate > 3].index, size=int(num_rows*0.3*0.2))
taxi_rides["category_driver_feedback"][category_driver_good_feedback_idx] = np.random.choice(driver_fb_cat_good, size=int(num_rows*0.3*0.2))

category_driver_bad_feedback_idx = np.random.choice(taxi_rides[taxi_rides.driver_rate < 4].index, size=int(num_rows*0.3*0.2))
taxi_rides["category_driver_feedback"][category_driver_bad_feedback_idx] = np.random.choice(driver_fb_cat_bad, size=int(num_rows*0.3*0.2))


text_good_feedback_driver_length = np.random.randint(low=0, high=len(driver_fb_cat_good), size=int(num_rows*0.3*0.2))
text_good_feedback_driver_sample = [random.sample(driver_fb_cat_good, i) for i in text_good_feedback_driver_length]
taxi_rides['text_driver_feedback'][category_driver_good_feedback_idx] = text_good_feedback_driver_sample

text_bad_feedback_driver_length = np.random.randint(low=0, high=len(driver_fb_cat_bad), size=int(num_rows*0.3*0.2))
text_bad_feedback_driver_sample = [random.sample(driver_fb_cat_bad, i) for i in text_bad_feedback_driver_length]
taxi_rides['text_driver_feedback'][category_driver_bad_feedback_idx] = text_bad_feedback_driver_sample

# ------------------

category_client_good_feedback_idx = np.random.choice(taxi_rides[taxi_rides.client_rate > 3].index, size=int(num_rows*0.3*0.2))
taxi_rides["category_client_feedback"][category_client_good_feedback_idx] = np.random.choice(client_fb_cat_good, size=int(num_rows*0.3*0.2))

category_client_bad_feedback_idx = np.random.choice(taxi_rides[taxi_rides.client_rate < 4].index, size=int(num_rows*0.3*0.2))
taxi_rides["category_client_feedback"][category_client_bad_feedback_idx] = np.random.choice(client_fb_cat_bad, size=int(num_rows*0.3*0.2))


text_good_feedback_client_length = np.random.randint(low=0, high=len(client_fb_cat_good), size=int(num_rows*0.3*0.2))
text_good_feedback_client_sample = [random.sample(client_fb_cat_good, i) for i in text_good_feedback_client_length]
taxi_rides['text_client_feedback'][category_client_good_feedback_idx] = text_good_feedback_client_sample

text_bad_feedback_client_length = np.random.randint(low=0, high=len(client_fb_cat_bad), size=int(num_rows*0.3*0.2))
text_bad_feedback_client_sample = [random.sample(client_fb_cat_bad, i) for i in text_bad_feedback_client_length]
taxi_rides['text_client_feedback'][category_client_bad_feedback_idx] = text_bad_feedback_client_sample


In [8]:
taxi_rides

,driver_id,client_id,start_point,end_point,distance,start_time,end_time,road_time,cost,driver_rate,client_rate,category_driver_feedback,text_driver_feedback,category_client_feedback,text_client_feedback
0,2035,126,"(51.62878, -0.172348)","(51.589973, -0.112392)",5.99,2023-04-06 07:37:33,2023-04-06 07:44:44.280,0 days 00:07:11.280000,8.00,5,5,expert navigation,[good service],quiet,[quiet]
1,3957,1044,"(51.551131, -0.173485)","(51.521158, -0.161323)",3.44,2023-04-29 03:39:50,2023-04-29 03:43:57.680,0 days 00:04:07.680000,6.72,4,4,good service,[],polite,"[polite, quiet]"
2,3270,1076,"(51.580247, -0.1188)","(51.651933, -0.077091)",8.48,2023-01-02 23:19:18,2023-01-02 23:29:28.560,0 days 00:10:10.560000,9.24,5,5,good service,"[expert navigation, nice car]",quiet,[pleasant]
3,2406,1021,"(51.546822, -0.442101)","(51.495447, -0.192481)",18.24,2023-02-02 07:30:58,2023-02-02 07:52:51.280,0 days 00:21:53.280000,14.12,5,5,expert navigation,[],quiet,"[pleasant, polite]"
4,3453,1974,"(51.566182, -0.137344)","(51.61868, -0.104788)",6.26,2023-10-19 15:08:33,2023-10-19 15:16:03.720,0 days 00:07:30.720000,8.13,1,1,bad car,[],loud,[loud]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,3062,674,"(51.611234, -0.423317)","(51.442887, -0.145358)",26.89,2023-09-04 10:36:11,2023-09-04 11:08:27.080,0 days 00:32:16.080000,27.67,NaN,NaN,NaN,NaN,NaN,NaN
9999996,3382,795,"(51.474597, -0.120644)","(51.425729, -0.218363)",8.70,2023-08-11 13:55:20,2023-08-11 14:05:46.400,0 days 00:10:26.400000,9.35,NaN,NaN,NaN,NaN,NaN,NaN
9999997,4413,37,"(51.524137, -0.177031)","(51.587977, -0.221811)",7.75,2023-09-21 23:01:29,2023-09-21 23:10:47.000,0 days 00:09:18,8.88,NaN,NaN,NaN,NaN,NaN,NaN
9999998,2313,1096,"(51.652474, -0.107021)","(51.484753, -0.183628)",19.40,2023-04-11 19:42:35,2023-04-11 20:05:51.800,0 days 00:23:16.800000,22.05,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
taxi_rides.to_csv('taxi.csv',  index=False)